# k-Abelian Complexity of Fibonacci sequence from 1 to 8

## Initial Numeration System Setup

First we define the subsitution, its numeration system and its fixpoint
sequence.

In [2]:
%%python
from licofage.kit import *
import os
setparams(True, True, os.environ["WALNUT_HOME"])

s = subst('01/0')
ns = address(s, "fib")
ns.gen_ns()
ns.gen_word_automaton()

>>> Address automaton for subst(0->01, 1->0)
  DT polynomial: X^2-X-1 (n=2)
  θ=1.618033988749895
  DT automaton: 2 states, 2 final states, 3 transitions.

>>> address(subst(0->01, 1->0), 'fib') is generating it's NS
  Numeration system automaton: 2 states, 2 final states, 3 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.ftvsMYWcSS/Walnut/Custom Bases/msd_fib.txt in format Walnut
  +address(subst(0->01, 1->0), 'fib')+address(subst(0->01, 1->0), 'fib')-address(subst(0->01, 1->0), 'fib') is generating it's dfa under name msd_fib_addition
  Combination polynomial is X^2-X-1
  θ=1.618033988749895
  Combination automaton: 8 states, 8 final states, 27 transitions.
  (P,u) vector bound: (0.0, 0.0) + (3.5124611797498106, 4.959674775249768)*C
  Fast poly bound: C in [(-1, -1), (2, 2)] => (14.049844718999243, 19.838699100999072)
Expanding DFA...done.
  Flattened automaton : 280 states, 11 final states, 623 transitions.
  Trimmed flattened automaton : 22 states, 11 fi

Then we setup a factor comparison predicate in Walnut and a first factor
occurence predicate.

In [3]:
def cut "?msd_fib i<=u & j<=v & u+j=v+i & u<n+i & v<n+j":
def feq_fib "?msd_fib ~(Eu,v $cut(i,j,n,u,v) & Fib[u]!=Fib[v])":
eval comp_fib n "?msd_fib Aj $feq_fib(i,j,n) => i<=j":

i<=u:6 states - 6ms
 j<=v:6 states - 0ms
  (i<=u&j<=v):36 states - 2ms
   (u+j)=(v+i):66 states - 9ms
    ((i<=u&j<=v)&(u+j)=(v+i)):70 states - 2ms
     u<(n+i):29 states - 2ms
      (((i<=u&j<=v)&(u+j)=(v+i))&u<(n+i)):317 states - 12ms
       v<(n+j):29 states - 2ms
        ((((i<=u&j<=v)&(u+j)=(v+i))&u<(n+i))&v<(n+j)):317 states - 9ms
Total computation time: 50ms.

Fib[u]!=Fib[v]:4 states - 0ms
 (cut(i,j,n,u,v))&Fib[u]!=Fib[v]):317 states - 5ms
  (E u , v (cut(i,j,n,u,v))&Fib[u]!=Fib[v])):23 states - 18ms
   ~(E u , v (cut(i,j,n,u,v))&Fib[u]!=Fib[v])):11 states - 1ms
Total computation time: 30ms.

i<=j:6 states - 0ms
 (feq_fib(i,j,n))=>i<=j):25 states - 2ms
  (A j (feq_fib(i,j,n))=>i<=j)):10 states - 1ms
Total computation time: 4ms.


From there we can define the boundary condition `bordercond`, as
explained in Fici-Puzynina-2023 section 8.1.

In [4]:
def bordercond "?msd_fib (k<=n => $feq_fib(i,j,k-1)) & (n<k => $feq_fib(i,j,n))":

k<=n:6 states - 0ms
computed ~:1 states - 12ms
computed ~:2 states - 0ms
 (k<=n=>feq_fib(i,j,(k-1)))):61 states - 3ms
  n<k:6 states - 0ms
   (n<k=>feq_fib(i,j,n))):61 states - 9ms
    ((k<=n=>feq_fib(i,j,(k-1))))&(n<k=>feq_fib(i,j,n)))):44 states - 1ms
Total computation time: 27ms.


## 1-abelian complexity

Define the 1-block map of fib and construct the conversion predicate
between both numeration systems.

In [5]:
%%python
s1 = s
ns1 = ns

### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Fib.

In [6]:
%%python
for (i,a) in enumerate(ns1.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s1, ns1.ns, **w)
    (parikh - ns1).gen_dfa(f"fibb1p{i}")

>>> Address automaton for subst(0->01, 1->0)
  DT polynomial: X^2-X-1 (n=2)
  θ=1.618033988749895
  DT automaton: 2 states, 2 final states, 3 transitions.

>>> +address(subst(0->01, 1->0), 'fib', '_': 0, '0': 1)-address(subst(0->01, 1->0), 'fib') is generating it's dfa under name fibb1p0
  Combination polynomial is X^2-X-1
  θ=1.618033988749895
  Combination automaton: 4 states, 4 final states, 9 transitions.
  (P,u) vector bound: (0.0, 0.0) + (3.5124611797498106, 4.959674775249768)*C
  Fast poly bound: C in [(-1, -1), (0, 1)] => (7.024922359499621, 9.919349550499536)
Expanding DFA...done.
  Flattened automaton : 58 states, 4 final states, 83 transitions.
  Trimmed flattened automaton : 6 states, 4 final states, 9 transitions.
  Real bound is [1, 1] (vs [7.024922359499621, 9.919349550499536]).
  After minimization: 5 states, 3 final states, 8 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.ftvsMYWcSS/Walnut/Automata Library/fibb1p0.txt in format Walnut

>>> 

### Compute the complexity function

Use Walnut to generate a linear representation for the 1-abelian
complexity.

In [7]:
def fac0 "?msd_fib Ex,y $fibb1p0(i,x) & $fibb1p0(i+n,y) & z+x=y":
def fac1 "?msd_fib Ex,y $fibb1p1(i,x) & $fibb1p1(i+n,y) & z+x=y":

(fibb1p0(i,x))&fibb1p0((i+n),y))):42 states - 1ms
 (z+x)=y:16 states - 0ms
  ((fibb1p0(i,x))&fibb1p0((i+n),y)))&(z+x)=y):65 states - 2ms
   (E x , y ((fibb1p0(i,x))&fibb1p0((i+n),y)))&(z+x)=y)):22 states - 0ms
Total computation time: 4ms.

(fibb1p1(i,x))&fibb1p1((i+n),y))):53 states - 0ms
 (z+x)=y:16 states - 0ms
  ((fibb1p1(i,x))&fibb1p1((i+n),y)))&(z+x)=y):102 states - 1ms
   (E x , y ((fibb1p1(i,x))&fibb1p1((i+n),y)))&(z+x)=y)):26 states - 0ms
Total computation time: 2ms.


In [8]:
def min0 "?msd_fib Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_fib Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":

y>=x:6 states - 0ms
 (fac0(j,n,y))=>y>=x):88 states - 3ms
  (A j , y (fac0(j,n,y))=>y>=x)):8 states - 3ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):15 states - 0ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):6 states - 0ms
Total computation time: 7ms.

y>=x:6 states - 0ms
 (fac1(j,n,y))=>y>=x):117 states - 5ms
  (A j , y (fac1(j,n,y))=>y>=x)):9 states - 4ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):18 states - 0ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):7 states - 0ms
Total computation time: 9ms.


In [9]:
def diff0 "?msd_fib Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_fib Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):32 states - 0ms
 (z+x)=y:16 states - 1ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):32 states - 0ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):14 states - 0ms
Total computation time: 1ms.

(min1(n,x))&fac1(i,n,y))):36 states - 0ms
 (z+x)=y:16 states - 0ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):36 states - 0ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):14 states - 0ms
Total computation time: 1ms.


In [10]:
def abeq_fib1 "?msd_fib (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z))":

(diff0(i,n,z))&diff0(j,n,z))):44 states - 1ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):35 states - 0ms
  (diff1(i,n,z))&diff1(j,n,z))):44 states - 0ms
   (E z (diff1(i,n,z))&diff1(j,n,z)))):35 states - 0ms
    ((E z (diff0(i,n,z))&diff0(j,n,z))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):35 states - 1ms
Total computation time: 2ms.


In [11]:
eval comp_fib1 n "?msd_fib Aj $abeq_fib1(i,j,n) => i<=j":

i<=j:6 states - 0ms
 (abeq_fib1(i,j,n))=>i<=j):52 states - 1ms
  (A j (abeq_fib1(i,j,n))=>i<=j)):4 states - 2ms
Total computation time: 3ms.


### Apply the semigroup trick

In [12]:
%SGT comp_fib1 msd_fib Comp_fib1

Reading Result/comp_fib1.mpl
Rank: 3
4 states
values: [0, 1, 2]
Writing Word Automata Library/Comp_fib1.txt
Done.

## 2-abelian complexity

Define the 2-block map of fib and construct the conversion predicate
between both numeration systems.

In [13]:
%%python
s2 = block(s, 2)
ns2 = address(s2, "fibb2")
ns2.gen_ns()
(ns-ns2).gen_dfa("conv_fib_fibb2")

>>> Address automaton for block(subst(0->01, 1->0), 2)
  DT polynomial: X^2-X-1 (n=2)
  θ=1.618033988749895
  DT automaton: 3 states, 3 final states, 5 transitions.

>>> address(block(subst(0->01, 1->0), 2), 'fibb2') is generating it's NS
  Numeration system automaton: 2 states, 2 final states, 3 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.ftvsMYWcSS/Walnut/Custom Bases/msd_fibb2.txt in format Walnut
  +address(block(subst(0->01, 1->0), 2), 'fibb2')+address(block(subst(0->01, 1->0), 2), 'fibb2')-address(block(subst(0->01, 1->0), 2), 'fibb2') is generating it's dfa under name msd_fibb2_addition
  Combination polynomial is X^2-X-1
  θ=1.618033988749895
  Combination automaton: 27 states, 27 final states, 125 transitions.
  (P,u) vector bound: (0.0, 0.0) + (3.5124611797498106, 4.959674775249768)*C
  Fast poly bound: C in [(-1, -1), (2, 2)] => (14.049844718999243, 19.838699100999072)
Expanding DFA...done.
  Flattened automaton : 623 states, 24 final states, 

Translate the border condition predicate into the current numeration
system.

In [14]:
def bordercond2 "?msd_fibb2 (?msd_fib Eii,jj,kk,nn 
($conv_fib_fibb2(?msd_fib ii, ?msd_fibb2 i) & 
$conv_fib_fibb2(?msd_fib jj, ?msd_fibb2 j) & 
$conv_fib_fibb2(?msd_fib kk, ?msd_fibb2 k) & 
$conv_fib_fibb2(?msd_fib nn, ?msd_fibb2 n) & 
$bordercond(ii,jj,kk,nn)))":






(conv_fib_fibb2(ii,i))&conv_fib_fibb2(jj,j))):4 states - 1ms
 ((conv_fib_fibb2(ii,i))&conv_fib_fibb2(jj,j)))&conv_fib_fibb2(kk,k))):8 states - 0ms
  (((conv_fib_fibb2(ii,i))&conv_fib_fibb2(jj,j)))&conv_fib_fibb2(kk,k)))&conv_fib_fibb2(nn,n))):16 states - 0ms
   ((((conv_fib_fibb2(ii,i))&conv_fib_fibb2(jj,j)))&conv_fib_fibb2(kk,k)))&conv_fib_fibb2(nn,n)))&bordercond(ii,jj,kk,nn))):44 states - 1ms
    (E ii , jj , kk , nn ((((conv_fib_fibb2(ii,i))&conv_fib_fibb2(jj,j)))&conv_fib_fibb2(kk,k)))&conv_fib_fibb2(nn,n)))&bordercond(ii,jj,kk,nn)))):44 states - 0ms
Total computation time: 4ms.


### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Fibb2.

In [15]:
%%python
for (i,a) in enumerate(ns2.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s2, ns2.ns, **w)
    (parikh - ns2).gen_dfa(f"fibb2p{i}")

>>> Address automaton for block(subst(0->01, 1->0), 2)
  DT polynomial: X^3-X^2-X (n=3)
  θ=1.618033988749895 (X^1)
  DT automaton: 3 states, 3 final states, 5 transitions.

>>> +address(block(subst(0->01, 1->0), 2), 'fibb2', '_': 0, 'a': 1)-address(block(subst(0->01, 1->0), 2), 'fibb2') is generating it's dfa under name fibb2p0
  Combination polynomial is X^3-X^2-X
  θ=1.618033988749895 (X^1)
  Combination automaton: 9 states, 9 final states, 25 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0) + (6.512461179749812, 8.748529157249603, 12.366563145999498)*C
  Fast poly bound: C in [(-1, -1, -1), (0, 0, 1)] => (19.537383539249436, 26.24558747174881, 37.09968943799849)
Expanding DFA...done.
  Flattened automaton : 302 states, 9 final states, 493 transitions.
  Trimmed flattened automaton : 16 states, 9 final states, 25 transitions.
  Real bound is [1, 1, 2] (vs [19.537383539249436, 26.24558747174881, 37.09968943799849]).
  After minimization: 9 states, 5 final states, 14 transitions.
  

### Compute the complexity function

Use Walnut to generate a linear representation for the 2-abelian
complexity.

In [16]:
def fac0 "?msd_fibb2 Ex,y $fibb2p0(i,x) & $fibb2p0(i+n,y) & z+x=y":
def fac1 "?msd_fibb2 Ex,y $fibb2p1(i,x) & $fibb2p1(i+n,y) & z+x=y":
def fac2 "?msd_fibb2 Ex,y $fibb2p2(i,x) & $fibb2p2(i+n,y) & z+x=y":

(fibb2p0(i,x))&fibb2p0((i+n),y))):80 states - 0ms
 (z+x)=y:16 states - 0ms
  ((fibb2p0(i,x))&fibb2p0((i+n),y)))&(z+x)=y):143 states - 2ms
   (E x , y ((fibb2p0(i,x))&fibb2p0((i+n),y)))&(z+x)=y)):33 states - 0ms
Total computation time: 3ms.

(fibb2p1(i,x))&fibb2p1((i+n),y))):53 states - 0ms
 (z+x)=y:16 states - 1ms
  ((fibb2p1(i,x))&fibb2p1((i+n),y)))&(z+x)=y):102 states - 1ms
   (E x , y ((fibb2p1(i,x))&fibb2p1((i+n),y)))&(z+x)=y)):26 states - 0ms
Total computation time: 2ms.

(fibb2p2(i,x))&fibb2p2((i+n),y))):104 states - 0ms
 (z+x)=y:16 states - 0ms
  ((fibb2p2(i,x))&fibb2p2((i+n),y)))&(z+x)=y):246 states - 2ms
   (E x , y ((fibb2p2(i,x))&fibb2p2((i+n),y)))&(z+x)=y)):50 states - 1ms
Total computation time: 5ms.


In [17]:
def min0 "?msd_fibb2 Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_fibb2 Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_fibb2 Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":

y>=x:6 states - 1ms
 (fac0(j,n,y))=>y>=x):138 states - 7ms
  (A j , y (fac0(j,n,y))=>y>=x)):9 states - 8ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):23 states - 0ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):7 states - 1ms
Total computation time: 17ms.

y>=x:6 states - 0ms
 (fac1(j,n,y))=>y>=x):117 states - 6ms
  (A j , y (fac1(j,n,y))=>y>=x)):9 states - 5ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):18 states - 1ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):7 states - 0ms
Total computation time: 12ms.

y>=x:6 states - 0ms
 (fac2(j,n,y))=>y>=x):212 states - 9ms
  (A j , y (fac2(j,n,y))=>y>=x)):12 states - 11ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):32 states - 0ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):11 states - 0ms
Total computation time: 21ms.


In [18]:
def diff0 "?msd_fibb2 Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_fibb2 Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_fibb2 Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):45 states - 0ms
 (z+x)=y:16 states - 1ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):44 states - 0ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):16 states - 0ms
Total computation time: 1ms.

(min1(n,x))&fac1(i,n,y))):36 states - 0ms
 (z+x)=y:16 states - 1ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):36 states - 0ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):14 states - 1ms
Total computation time: 2ms.

(min2(n,x))&fac2(i,n,y))):65 states - 0ms
 (z+x)=y:16 states - 0ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):66 states - 1ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):19 states - 0ms
Total computation time: 1ms.


In [19]:
def abeq_fibb2 "?msd_fibb2 $bordercond2(i,j,2,n+1) & (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z))":
def abeq_fib2 "?msd_fib (n<1 & $feq_fib(i,j,n)) | (n>=1 & (?msd_fibb2 Eii,jj,nn ($conv_fib_fibb2(?msd_fib i, ?msd_fibb2 ii) & $conv_fib_fibb2(?msd_fib j, ?msd_fibb2 jj) & $conv_fib_fibb2(?msd_fib n, ?msd_fibb2 nn) & $abeq_fibb2(ii,jj,nn-1))))":

computed ~:1 states - 0ms
computed ~:2 states - 1ms
(diff0(i,n,z))&diff0(j,n,z))):60 states - 0ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):51 states - 1ms
  (bordercond2(i,j,2,(n+1)))&(E z (diff0(i,n,z))&diff0(j,n,z))))):51 states - 0ms
   (diff1(i,n,z))&diff1(j,n,z))):44 states - 1ms
    (E z (diff1(i,n,z))&diff1(j,n,z)))):35 states - 0ms
     ((bordercond2(i,j,2,(n+1)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):51 states - 1ms
      (diff2(i,n,z))&diff2(j,n,z))):65 states - 0ms
       (E z (diff2(i,n,z))&diff2(j,n,z)))):51 states - 1ms
        (((bordercond2(i,j,2,(n+1)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):51 states - 0ms
Total computation time: 6ms.

n<1:1 states - 0ms
 (n<1&feq_fib(i,j,n))):4 states - 0ms
  n>=1:3 states - 0ms
   (conv_fib_fibb2(i,ii))&conv_fib_fibb2(j,jj))):4 states - 0ms
    ((conv_fib_fibb2(i,ii))&conv_fib_fibb2(j,jj)))&conv_fib_fibb2(n,nn))):8 states - 0ms
  

In [20]:
eval comp_fib2 n "?msd_fib Aj $abeq_fib2(i,j,n) => i<=j":

i<=j:6 states - 0ms
 (abeq_fib2(i,j,n))=>i<=j):87 states - 3ms
  (A j (abeq_fib2(i,j,n))=>i<=j)):12 states - 2ms
Total computation time: 5ms.


### Apply the semigroup trick

In [21]:
%SGT comp_fib2 msd_fib Comp_fib2

Reading Result/comp_fib2.mpl
Rank: 5
6 states
values: [0, 1, 2, 3, 4]
Writing Word Automata Library/Comp_fib2.txt
Done.

## 3-abelian complexity

Define the 3-block map of fib and construct the conversion predicate
between both numeration systems.

In [22]:
%%python
s3 = block(s, 3)
ns3 = address(s3, "fibb3")
ns3.gen_ns()
(ns-ns3).gen_dfa("conv_fib_fibb3")

>>> Address automaton for block(subst(0->01, 1->0), 3)
  DT polynomial: X^2-X-1 (n=2)
  θ=1.618033988749895
  DT automaton: 4 states, 4 final states, 6 transitions.

>>> address(block(subst(0->01, 1->0), 3), 'fibb3') is generating it's NS
  Numeration system automaton: 2 states, 2 final states, 3 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.ftvsMYWcSS/Walnut/Custom Bases/msd_fibb3.txt in format Walnut
  +address(block(subst(0->01, 1->0), 3), 'fibb3')+address(block(subst(0->01, 1->0), 3), 'fibb3')-address(block(subst(0->01, 1->0), 3), 'fibb3') is generating it's dfa under name msd_fibb3_addition
  Combination polynomial is X^2-X-1
  θ=1.618033988749895
  Combination automaton: 64 states, 64 final states, 216 transitions.
  (P,u) vector bound: (0.0, 0.0) + (3.5124611797498106, 4.959674775249768)*C
  Fast poly bound: C in [(-1, -1), (2, 2)] => (14.049844718999243, 19.838699100999072)
Expanding DFA...done.
  Flattened automaton : 1092 states, 42 final states,

Translate the border condition predicate into the current numeration
system.

In [23]:
def bordercond3 "?msd_fibb3 (?msd_fib Eii,jj,kk,nn 
($conv_fib_fibb3(?msd_fib ii, ?msd_fibb3 i) & 
$conv_fib_fibb3(?msd_fib jj, ?msd_fibb3 j) & 
$conv_fib_fibb3(?msd_fib kk, ?msd_fibb3 k) & 
$conv_fib_fibb3(?msd_fib nn, ?msd_fibb3 n) & 
$bordercond(ii,jj,kk,nn)))":






(conv_fib_fibb3(ii,i))&conv_fib_fibb3(jj,j))):4 states - 0ms
 ((conv_fib_fibb3(ii,i))&conv_fib_fibb3(jj,j)))&conv_fib_fibb3(kk,k))):8 states - 0ms
  (((conv_fib_fibb3(ii,i))&conv_fib_fibb3(jj,j)))&conv_fib_fibb3(kk,k)))&conv_fib_fibb3(nn,n))):16 states - 0ms
   ((((conv_fib_fibb3(ii,i))&conv_fib_fibb3(jj,j)))&conv_fib_fibb3(kk,k)))&conv_fib_fibb3(nn,n)))&bordercond(ii,jj,kk,nn))):44 states - 1ms
    (E ii , jj , kk , nn ((((conv_fib_fibb3(ii,i))&conv_fib_fibb3(jj,j)))&conv_fib_fibb3(kk,k)))&conv_fib_fibb3(nn,n)))&bordercond(ii,jj,kk,nn)))):44 states - 1ms
Total computation time: 3ms.


### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Fibb3.

In [24]:
%%python
for (i,a) in enumerate(ns3.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s3, ns3.ns, **w)
    (parikh - ns3).gen_dfa(f"fibb3p{i}")

>>> Address automaton for block(subst(0->01, 1->0), 3)
  DT polynomial: X^3-X^2-X (n=3)
  θ=1.618033988749895 (X^1)
  DT automaton: 4 states, 4 final states, 6 transitions.

>>> +address(block(subst(0->01, 1->0), 3), 'fibb3', '_': 0, 'a': 1)-address(block(subst(0->01, 1->0), 3), 'fibb3') is generating it's dfa under name fibb3p0
  Combination polynomial is X^3-X^2-X
  θ=1.618033988749895 (X^1)
  Combination automaton: 16 states, 16 final states, 36 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0) + (6.512461179749812, 8.748529157249603, 12.366563145999498)*C
  Fast poly bound: C in [(-1, -1, -1), (0, 0, 1)] => (19.537383539249436, 26.24558747174881, 37.09968943799849)
Expanding DFA...done.
  Flattened automaton : 398 states, 12 final states, 585 transitions.
  Trimmed flattened automaton : 20 states, 12 final states, 30 transitions.
  Real bound is [1, 1, 2] (vs [19.537383539249436, 26.24558747174881, 37.09968943799849]).
  After minimization: 9 states, 5 final states, 14 transitions

### Compute the complexity function

Use Walnut to generate a linear representation for the 3-abelian
complexity.

In [25]:
def fac0 "?msd_fibb3 Ex,y $fibb3p0(i,x) & $fibb3p0(i+n,y) & z+x=y":
def fac1 "?msd_fibb3 Ex,y $fibb3p1(i,x) & $fibb3p1(i+n,y) & z+x=y":
def fac2 "?msd_fibb3 Ex,y $fibb3p2(i,x) & $fibb3p2(i+n,y) & z+x=y":
def fac3 "?msd_fibb3 Ex,y $fibb3p3(i,x) & $fibb3p3(i+n,y) & z+x=y":

(fibb3p0(i,x))&fibb3p0((i+n),y))):80 states - 1ms
 (z+x)=y:16 states - 0ms
  ((fibb3p0(i,x))&fibb3p0((i+n),y)))&(z+x)=y):143 states - 3ms
   (E x , y ((fibb3p0(i,x))&fibb3p0((i+n),y)))&(z+x)=y)):33 states - 0ms
Total computation time: 5ms.

(fibb3p1(i,x))&fibb3p1((i+n),y))):134 states - 3ms
 (z+x)=y:16 states - 0ms
  ((fibb3p1(i,x))&fibb3p1((i+n),y)))&(z+x)=y):279 states - 2ms
   (E x , y ((fibb3p1(i,x))&fibb3p1((i+n),y)))&(z+x)=y)):71 states - 0ms
Total computation time: 5ms.

(fibb3p2(i,x))&fibb3p2((i+n),y))):104 states - 0ms
 (z+x)=y:16 states - 0ms
  ((fibb3p2(i,x))&fibb3p2((i+n),y)))&(z+x)=y):246 states - 3ms
   (E x , y ((fibb3p2(i,x))&fibb3p2((i+n),y)))&(z+x)=y)):50 states - 1ms
Total computation time: 5ms.

(fibb3p3(i,x))&fibb3p3((i+n),y))):207 states - 1ms
 (z+x)=y:16 states - 0ms
  ((fibb3p3(i,x))&fibb3p3((i+n),y)))&(z+x)=y):514 states - 4ms
   (E x , y ((fibb3p3(i,x))&fibb3p3((i+n),y)))&(z+x)=y)):106 states - 1ms
Total computation time: 7ms.


In [26]:
def min0 "?msd_fibb3 Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_fibb3 Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_fibb3 Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":
def min3 "?msd_fibb3 Ei $fac3(i,n,x) & Aj,y $fac3(j,n,y) => y>=x":

y>=x:6 states - 0ms
 (fac0(j,n,y))=>y>=x):138 states - 4ms
  (A j , y (fac0(j,n,y))=>y>=x)):9 states - 4ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):23 states - 0ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):7 states - 0ms
Total computation time: 8ms.

y>=x:6 states - 0ms
 (fac1(j,n,y))=>y>=x):267 states - 8ms
  (A j , y (fac1(j,n,y))=>y>=x)):12 states - 10ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):43 states - 0ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):11 states - 1ms
Total computation time: 20ms.

y>=x:6 states - 0ms
 (fac2(j,n,y))=>y>=x):212 states - 7ms
  (A j , y (fac2(j,n,y))=>y>=x)):12 states - 6ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):32 states - 0ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):11 states - 0ms
Total computation time: 13ms.

y>=x:6 states - 0ms
 (fac3(j,n,y))=>y>=x):387 states - 12ms
  (A j , y (fac3(j,n,y))=>y>=x)):17 states - 13ms
   (fac3(i,n,x))&(A j , y (fac3(j,n,y))=>y>=x))):66 states - 0ms

In [27]:
def diff0 "?msd_fibb3 Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_fibb3 Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_fibb3 Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":
def diff3 "?msd_fibb3 Ex,y $min3(n,x) & $fac3(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):45 states - 0ms
 (z+x)=y:16 states - 1ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):44 states - 0ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):16 states - 0ms
Total computation time: 1ms.

(min1(n,x))&fac1(i,n,y))):86 states - 0ms
 (z+x)=y:16 states - 0ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):81 states - 1ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):26 states - 0ms
Total computation time: 1ms.

(min2(n,x))&fac2(i,n,y))):65 states - 0ms
 (z+x)=y:16 states - 0ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):66 states - 0ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):19 states - 0ms
Total computation time: 1ms.

(min3(n,x))&fac3(i,n,y))):134 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min3(n,x))&fac3(i,n,y)))&(z+x)=y):130 states - 0ms
   (E x , y ((min3(n,x))&fac3(i,n,y)))&(z+x)=y)):33 states - 0ms
Total computation time: 2ms.


In [28]:
def abeq_fibb3 "?msd_fibb3 $bordercond3(i,j,3,n+2) & (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z)) & (Ez $diff3(i,n,z) &  $diff3(j,n,z))":
def abeq_fib3 "?msd_fib (n<2 & $feq_fib(i,j,n)) | (n>=2 & (?msd_fibb3 Eii,jj,nn ($conv_fib_fibb3(?msd_fib i, ?msd_fibb3 ii) & $conv_fib_fibb3(?msd_fib j, ?msd_fibb3 jj) & $conv_fib_fibb3(?msd_fib n, ?msd_fibb3 nn) & $abeq_fibb3(ii,jj,nn-2))))":

computed ~:1 states - 1ms
computed ~:2 states - 0ms
(diff0(i,n,z))&diff0(j,n,z))):60 states - 0ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):51 states - 1ms
  (bordercond3(i,j,3,(n+2)))&(E z (diff0(i,n,z))&diff0(j,n,z))))):59 states - 0ms
   (diff1(i,n,z))&diff1(j,n,z))):106 states - 1ms
    (E z (diff1(i,n,z))&diff1(j,n,z)))):102 states - 0ms
     ((bordercond3(i,j,3,(n+2)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):67 states - 1ms
      (diff2(i,n,z))&diff2(j,n,z))):65 states - 0ms
       (E z (diff2(i,n,z))&diff2(j,n,z)))):51 states - 0ms
        (((bordercond3(i,j,3,(n+2)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):59 states - 1ms
         (diff3(i,n,z))&diff3(j,n,z))):117 states - 0ms
          (E z (diff3(i,n,z))&diff3(j,n,z)))):96 states - 1ms
           ((((bordercond3(i,j,3,(n+2)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n

In [29]:
eval comp_fib3 n "?msd_fib Aj $abeq_fib3(i,j,n) => i<=j":

i<=j:6 states - 0ms
 (abeq_fib3(i,j,n))=>i<=j):102 states - 1ms
  (A j (abeq_fib3(i,j,n))=>i<=j)):17 states - 3ms
Total computation time: 6ms.


### Apply the semigroup trick

In [30]:
%SGT comp_fib3 msd_fib Comp_fib3

Reading Result/comp_fib3.mpl
Rank: 6
8 states
values: [0, 1, 2, 3, 4, 5, 6]
Writing Word Automata Library/Comp_fib3.txt
Done.

## 4-abelian complexity

Define the 4-block map of fib and construct the conversion predicate
between both numeration systems.

In [31]:
%%python
s4 = block(s, 4)
ns4 = address(s4, "fibb4")
ns4.gen_ns()
(ns-ns4).gen_dfa("conv_fib_fibb4")

>>> Address automaton for block(subst(0->01, 1->0), 4)
  DT polynomial: X^2-X-1 (n=2)
  θ=1.618033988749895
  DT automaton: 5 states, 5 final states, 8 transitions.

>>> address(block(subst(0->01, 1->0), 4), 'fibb4') is generating it's NS
  Numeration system automaton: 2 states, 2 final states, 3 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.ftvsMYWcSS/Walnut/Custom Bases/msd_fibb4.txt in format Walnut
  +address(block(subst(0->01, 1->0), 4), 'fibb4')+address(block(subst(0->01, 1->0), 4), 'fibb4')-address(block(subst(0->01, 1->0), 4), 'fibb4') is generating it's dfa under name msd_fibb4_addition
  Combination polynomial is X^2-X-1
  θ=1.618033988749895
  Combination automaton: 125 states, 125 final states, 512 transitions.
  (P,u) vector bound: (0.0, 0.0) + (3.5124611797498106, 4.959674775249768)*C
  Fast poly bound: C in [(-1, -1), (2, 2)] => (14.049844718999243, 19.838699100999072)
Expanding DFA...done.
  Flattened automaton : 1682 states, 65 final state

Translate the border condition predicate into the current numeration
system.

In [32]:
def bordercond4 "?msd_fibb4 (?msd_fib Eii,jj,kk,nn 
($conv_fib_fibb4(?msd_fib ii, ?msd_fibb4 i) & 
$conv_fib_fibb4(?msd_fib jj, ?msd_fibb4 j) & 
$conv_fib_fibb4(?msd_fib kk, ?msd_fibb4 k) & 
$conv_fib_fibb4(?msd_fib nn, ?msd_fibb4 n) & 
$bordercond(ii,jj,kk,nn)))":






(conv_fib_fibb4(ii,i))&conv_fib_fibb4(jj,j))):4 states - 0ms
 ((conv_fib_fibb4(ii,i))&conv_fib_fibb4(jj,j)))&conv_fib_fibb4(kk,k))):8 states - 0ms
  (((conv_fib_fibb4(ii,i))&conv_fib_fibb4(jj,j)))&conv_fib_fibb4(kk,k)))&conv_fib_fibb4(nn,n))):16 states - 0ms
   ((((conv_fib_fibb4(ii,i))&conv_fib_fibb4(jj,j)))&conv_fib_fibb4(kk,k)))&conv_fib_fibb4(nn,n)))&bordercond(ii,jj,kk,nn))):44 states - 1ms
    (E ii , jj , kk , nn ((((conv_fib_fibb4(ii,i))&conv_fib_fibb4(jj,j)))&conv_fib_fibb4(kk,k)))&conv_fib_fibb4(nn,n)))&bordercond(ii,jj,kk,nn)))):44 states - 1ms
Total computation time: 2ms.


### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Fibb4.

In [33]:
%%python
for (i,a) in enumerate(ns4.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s4, ns4.ns, **w)
    (parikh - ns4).gen_dfa(f"fibb4p{i}")

>>> Address automaton for block(subst(0->01, 1->0), 4)
  DT polynomial: X^4-X^3-X^2 (n=4)
  θ=1.618033988749895 (X^2)
  DT automaton: 5 states, 5 final states, 8 transitions.

>>> +address(block(subst(0->01, 1->0), 4), 'fibb4', '_': 0, 'a': 1)-address(block(subst(0->01, 1->0), 4), 'fibb4') is generating it's dfa under name fibb4p0
  Combination polynomial is X^4-X^3-X^2
  θ=1.618033988749895 (X^2)
  Combination automaton: 25 states, 25 final states, 64 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0, 0.0) + (10.577708763999668, 13.984597134749396, 19.49705831449921, 28.416407864998753)*C
  Fast poly bound: C in [(-1, -1, -1, -1), (0, 0, 1, 1)] => (42.31083505599867, 55.938388538997586, 77.98823325799684, 113.66563145999501)
Expanding DFA...done.
  Flattened automaton : 1378 states, 25 final states, 2180 transitions.
  Trimmed flattened automaton : 43 states, 25 final states, 66 transitions.
  Real bound is [1, 2, 3, 4] (vs [42.31083505599867, 55.938388538997586, 77.98823325799684, 11

### Compute the complexity function

Use Walnut to generate a linear representation for the 4-abelian
complexity.

In [34]:
def fac0 "?msd_fibb4 Ex,y $fibb4p0(i,x) & $fibb4p0(i+n,y) & z+x=y":
def fac1 "?msd_fibb4 Ex,y $fibb4p1(i,x) & $fibb4p1(i+n,y) & z+x=y":
def fac2 "?msd_fibb4 Ex,y $fibb4p2(i,x) & $fibb4p2(i+n,y) & z+x=y":
def fac3 "?msd_fibb4 Ex,y $fibb4p3(i,x) & $fibb4p3(i+n,y) & z+x=y":
def fac4 "?msd_fibb4 Ex,y $fibb4p4(i,x) & $fibb4p4(i+n,y) & z+x=y":

(fibb4p0(i,x))&fibb4p0((i+n),y))):164 states - 1ms
 (z+x)=y:16 states - 1ms
  ((fibb4p0(i,x))&fibb4p0((i+n),y)))&(z+x)=y):330 states - 3ms
   (E x , y ((fibb4p0(i,x))&fibb4p0((i+n),y)))&(z+x)=y)):93 states - 1ms
Total computation time: 11ms.

(fibb4p1(i,x))&fibb4p1((i+n),y))):134 states - 0ms
 (z+x)=y:16 states - 0ms
  ((fibb4p1(i,x))&fibb4p1((i+n),y)))&(z+x)=y):279 states - 2ms
   (E x , y ((fibb4p1(i,x))&fibb4p1((i+n),y)))&(z+x)=y)):71 states - 0ms
Total computation time: 3ms.

(fibb4p2(i,x))&fibb4p2((i+n),y))):104 states - 0ms
 (z+x)=y:16 states - 0ms
  ((fibb4p2(i,x))&fibb4p2((i+n),y)))&(z+x)=y):246 states - 1ms
   (E x , y ((fibb4p2(i,x))&fibb4p2((i+n),y)))&(z+x)=y)):50 states - 0ms
Total computation time: 2ms.

(fibb4p3(i,x))&fibb4p3((i+n),y))):252 states - 1ms
 (z+x)=y:16 states - 0ms
  ((fibb4p3(i,x))&fibb4p3((i+n),y)))&(z+x)=y):602 states - 4ms
   (E x , y ((fibb4p3(i,x))&fibb4p3((i+n),y)))&(z+x)=y)):142 states - 1ms
Total computation time: 6ms.

(fibb4p4(i,x))&fibb4p4((i+n),y

In [35]:
def min0 "?msd_fibb4 Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_fibb4 Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_fibb4 Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":
def min3 "?msd_fibb4 Ei $fac3(i,n,x) & Aj,y $fac3(j,n,y) => y>=x":
def min4 "?msd_fibb4 Ei $fac4(i,n,x) & Aj,y $fac4(j,n,y) => y>=x":

y>=x:6 states - 0ms
 (fac0(j,n,y))=>y>=x):339 states - 8ms
  (A j , y (fac0(j,n,y))=>y>=x)):12 states - 9ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):59 states - 0ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):11 states - 0ms
Total computation time: 18ms.

y>=x:6 states - 0ms
 (fac1(j,n,y))=>y>=x):267 states - 8ms
  (A j , y (fac1(j,n,y))=>y>=x)):12 states - 9ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):43 states - 0ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):11 states - 0ms
Total computation time: 18ms.

y>=x:6 states - 1ms
 (fac2(j,n,y))=>y>=x):212 states - 6ms
  (A j , y (fac2(j,n,y))=>y>=x)):12 states - 6ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):32 states - 0ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):11 states - 0ms
Total computation time: 13ms.

y>=x:6 states - 0ms
 (fac3(j,n,y))=>y>=x):503 states - 16ms
  (A j , y (fac3(j,n,y))=>y>=x)):17 states - 16ms
   (fac3(i,n,x))&(A j , y (fac3(j,n,y))=>y>=x))):84 states - 0

In [36]:
def diff0 "?msd_fibb4 Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_fibb4 Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_fibb4 Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":
def diff3 "?msd_fibb4 Ex,y $min3(n,x) & $fac3(i,n,y) & z+x=y":
def diff4 "?msd_fibb4 Ex,y $min4(n,x) & $fac4(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):113 states - 0ms
 (z+x)=y:16 states - 0ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):106 states - 1ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):35 states - 0ms
Total computation time: 2ms.

(min1(n,x))&fac1(i,n,y))):86 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):81 states - 0ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):26 states - 1ms
Total computation time: 2ms.

(min2(n,x))&fac2(i,n,y))):65 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):66 states - 0ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):19 states - 0ms
Total computation time: 1ms.

(min3(n,x))&fac3(i,n,y))):178 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min3(n,x))&fac3(i,n,y)))&(z+x)=y):166 states - 2ms
   (E x , y ((min3(n,x))&fac3(i,n,y)))&(z+x)=y)):41 states - 1ms
Total computation time: 5ms.

(min4(n,x))&fac4(i,n,y))):134 states - 1ms
 (z+x)=y:16 states - 1ms
  ((min4(n,x))&fac4(i,n,y)))&(z+x)=y):130 states - 2ms
   (E x ,

In [37]:
def abeq_fibb4 "?msd_fibb4 $bordercond4(i,j,4,n+3) & (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z)) & (Ez $diff3(i,n,z) &  $diff3(j,n,z)) & (Ez $diff4(i,n,z) &  $diff4(j,n,z))":
def abeq_fib4 "?msd_fib (n<3 & $feq_fib(i,j,n)) | (n>=3 & (?msd_fibb4 Eii,jj,nn ($conv_fib_fibb4(?msd_fib i, ?msd_fibb4 ii) & $conv_fib_fibb4(?msd_fib j, ?msd_fibb4 jj) & $conv_fib_fibb4(?msd_fib n, ?msd_fibb4 nn) & $abeq_fibb4(ii,jj,nn-3))))":

computed ~:1 states - 1ms
computed ~:2 states - 0ms
(diff0(i,n,z))&diff0(j,n,z))):195 states - 1ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):201 states - 1ms
  (bordercond4(i,j,4,(n+3)))&(E z (diff0(i,n,z))&diff0(j,n,z))))):89 states - 1ms
   (diff1(i,n,z))&diff1(j,n,z))):106 states - 1ms
    (E z (diff1(i,n,z))&diff1(j,n,z)))):102 states - 0ms
     ((bordercond4(i,j,4,(n+3)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):83 states - 1ms
      (diff2(i,n,z))&diff2(j,n,z))):65 states - 0ms
       (E z (diff2(i,n,z))&diff2(j,n,z)))):51 states - 0ms
        (((bordercond4(i,j,4,(n+3)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):69 states - 1ms
         (diff3(i,n,z))&diff3(j,n,z))):210 states - 1ms
          (E z (diff3(i,n,z))&diff3(j,n,z)))):195 states - 1ms
           ((((bordercond4(i,j,4,(n+3)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(

In [38]:
eval comp_fib4 n "?msd_fib Aj $abeq_fib4(i,j,n) => i<=j":

i<=j:6 states - 0ms
 (abeq_fib4(i,j,n))=>i<=j):116 states - 2ms
  (A j (abeq_fib4(i,j,n))=>i<=j)):23 states - 2ms
Total computation time: 4ms.


### Apply the semigroup trick

In [39]:
%SGT comp_fib4 msd_fib Comp_fib4

Reading Result/comp_fib4.mpl
Rank: 7
10 states
values: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Writing Word Automata Library/Comp_fib4.txt
Done.

## 5-abelian complexity

Define the 5-block map of fib and construct the conversion predicate
between both numeration systems.

In [40]:
%%python
s5 = block(s, 5)
ns5 = address(s5, "fibb5")
ns5.gen_ns()
(ns-ns5).gen_dfa("conv_fib_fibb5")

>>> Address automaton for block(subst(0->01, 1->0), 5)
  DT polynomial: X^2-X-1 (n=2)
  θ=1.618033988749895
  DT automaton: 6 states, 6 final states, 10 transitions.

>>> address(block(subst(0->01, 1->0), 5), 'fibb5') is generating it's NS
  Numeration system automaton: 2 states, 2 final states, 3 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.ftvsMYWcSS/Walnut/Custom Bases/msd_fibb5.txt in format Walnut
  +address(block(subst(0->01, 1->0), 5), 'fibb5')+address(block(subst(0->01, 1->0), 5), 'fibb5')-address(block(subst(0->01, 1->0), 5), 'fibb5') is generating it's dfa under name msd_fibb5_addition
  Combination polynomial is X^2-X-1
  θ=1.618033988749895
  Combination automaton: 216 states, 216 final states, 1000 transitions.
  (P,u) vector bound: (0.0, 0.0) + (3.5124611797498106, 4.959674775249768)*C
  Fast poly bound: C in [(-1, -1), (2, 2)] => (14.049844718999243, 19.838699100999072)
Expanding DFA...done.
  Flattened automaton : 2388 states, 93 final sta

Translate the border condition predicate into the current numeration
system.

In [41]:
def bordercond5 "?msd_fibb5 (?msd_fib Eii,jj,kk,nn 
($conv_fib_fibb5(?msd_fib ii, ?msd_fibb5 i) & 
$conv_fib_fibb5(?msd_fib jj, ?msd_fibb5 j) & 
$conv_fib_fibb5(?msd_fib kk, ?msd_fibb5 k) & 
$conv_fib_fibb5(?msd_fib nn, ?msd_fibb5 n) & 
$bordercond(ii,jj,kk,nn)))":






(conv_fib_fibb5(ii,i))&conv_fib_fibb5(jj,j))):4 states - 0ms
 ((conv_fib_fibb5(ii,i))&conv_fib_fibb5(jj,j)))&conv_fib_fibb5(kk,k))):8 states - 0ms
  (((conv_fib_fibb5(ii,i))&conv_fib_fibb5(jj,j)))&conv_fib_fibb5(kk,k)))&conv_fib_fibb5(nn,n))):16 states - 0ms
   ((((conv_fib_fibb5(ii,i))&conv_fib_fibb5(jj,j)))&conv_fib_fibb5(kk,k)))&conv_fib_fibb5(nn,n)))&bordercond(ii,jj,kk,nn))):44 states - 1ms
    (E ii , jj , kk , nn ((((conv_fib_fibb5(ii,i))&conv_fib_fibb5(jj,j)))&conv_fib_fibb5(kk,k)))&conv_fib_fibb5(nn,n)))&bordercond(ii,jj,kk,nn)))):44 states - 0ms
Total computation time: 2ms.


### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Fibb5.

In [42]:
%%python
for (i,a) in enumerate(ns5.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s5, ns5.ns, **w)
    (parikh - ns5).gen_dfa(f"fibb5p{i}")

>>> Address automaton for block(subst(0->01, 1->0), 5)
  DT polynomial: X^4-X^3-X^2 (n=4)
  θ=1.618033988749895 (X^2)
  DT automaton: 6 states, 6 final states, 10 transitions.

>>> +address(block(subst(0->01, 1->0), 5), 'fibb5', '_': 0, 'a': 1)-address(block(subst(0->01, 1->0), 5), 'fibb5') is generating it's dfa under name fibb5p0
  Combination polynomial is X^4-X^3-X^2
  θ=1.618033988749895 (X^2)
  Combination automaton: 36 states, 36 final states, 100 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0, 0.0) + (10.577708763999668, 13.984597134749396, 19.49705831449921, 28.416407864998753)*C
  Fast poly bound: C in [(-1, -1, -1, -1), (0, 0, 1, 1)] => (42.31083505599867, 55.938388538997586, 77.98823325799684, 113.66563145999501)
Expanding DFA...done.
  Flattened automaton : 1648 states, 28 final states, 2705 transitions.
  Trimmed flattened automaton : 51 states, 28 final states, 82 transitions.
  Real bound is [1, 2, 3, 4] (vs [42.31083505599867, 55.938388538997586, 77.98823325799684, 

### Compute the complexity function

Use Walnut to generate a linear representation for the 5-abelian
complexity.

In [43]:
def fac0 "?msd_fibb5 Ex,y $fibb5p0(i,x) & $fibb5p0(i+n,y) & z+x=y":
def fac1 "?msd_fibb5 Ex,y $fibb5p1(i,x) & $fibb5p1(i+n,y) & z+x=y":
def fac2 "?msd_fibb5 Ex,y $fibb5p2(i,x) & $fibb5p2(i+n,y) & z+x=y":
def fac3 "?msd_fibb5 Ex,y $fibb5p3(i,x) & $fibb5p3(i+n,y) & z+x=y":
def fac4 "?msd_fibb5 Ex,y $fibb5p4(i,x) & $fibb5p4(i+n,y) & z+x=y":
def fac5 "?msd_fibb5 Ex,y $fibb5p5(i,x) & $fibb5p5(i+n,y) & z+x=y":

(fibb5p0(i,x))&fibb5p0((i+n),y))):164 states - 0ms
 (z+x)=y:16 states - 0ms
  ((fibb5p0(i,x))&fibb5p0((i+n),y)))&(z+x)=y):330 states - 1ms
   (E x , y ((fibb5p0(i,x))&fibb5p0((i+n),y)))&(z+x)=y)):93 states - 1ms
Total computation time: 4ms.

(fibb5p1(i,x))&fibb5p1((i+n),y))):134 states - 0ms
 (z+x)=y:16 states - 0ms
  ((fibb5p1(i,x))&fibb5p1((i+n),y)))&(z+x)=y):279 states - 2ms
   (E x , y ((fibb5p1(i,x))&fibb5p1((i+n),y)))&(z+x)=y)):71 states - 1ms
Total computation time: 4ms.

(fibb5p2(i,x))&fibb5p2((i+n),y))):282 states - 1ms
 (z+x)=y:16 states - 0ms
  ((fibb5p2(i,x))&fibb5p2((i+n),y)))&(z+x)=y):626 states - 7ms
   (E x , y ((fibb5p2(i,x))&fibb5p2((i+n),y)))&(z+x)=y)):168 states - 1ms
Total computation time: 10ms.

(fibb5p3(i,x))&fibb5p3((i+n),y))):252 states - 0ms
 (z+x)=y:16 states - 1ms
  ((fibb5p3(i,x))&fibb5p3((i+n),y)))&(z+x)=y):602 states - 4ms
   (E x , y ((fibb5p3(i,x))&fibb5p3((i+n),y)))&(z+x)=y)):142 states - 1ms
Total computation time: 7ms.

(fibb5p4(i,x))&fibb5p4((i+n),

In [44]:
def min0 "?msd_fibb5 Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_fibb5 Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_fibb5 Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":
def min3 "?msd_fibb5 Ei $fac3(i,n,x) & Aj,y $fac3(j,n,y) => y>=x":
def min4 "?msd_fibb5 Ei $fac4(i,n,x) & Aj,y $fac4(j,n,y) => y>=x":
def min5 "?msd_fibb5 Ei $fac5(i,n,x) & Aj,y $fac5(j,n,y) => y>=x":

y>=x:6 states - 0ms
 (fac0(j,n,y))=>y>=x):339 states - 9ms
  (A j , y (fac0(j,n,y))=>y>=x)):12 states - 8ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):59 states - 1ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):11 states - 0ms
Total computation time: 19ms.

y>=x:6 states - 0ms
 (fac1(j,n,y))=>y>=x):267 states - 8ms
  (A j , y (fac1(j,n,y))=>y>=x)):12 states - 9ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):43 states - 1ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):11 states - 0ms
Total computation time: 19ms.

y>=x:6 states - 0ms
 (fac2(j,n,y))=>y>=x):566 states - 20ms
  (A j , y (fac2(j,n,y))=>y>=x)):17 states - 14ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):101 states - 0ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):17 states - 0ms
Total computation time: 36ms.

y>=x:6 states - 0ms
 (fac3(j,n,y))=>y>=x):503 states - 15ms
  (A j , y (fac3(j,n,y))=>y>=x)):17 states - 13ms
   (fac3(i,n,x))&(A j , y (fac3(j,n,y))=>y>=x))):84 states 

In [45]:
def diff0 "?msd_fibb5 Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_fibb5 Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_fibb5 Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":
def diff3 "?msd_fibb5 Ex,y $min3(n,x) & $fac3(i,n,y) & z+x=y":
def diff4 "?msd_fibb5 Ex,y $min4(n,x) & $fac4(i,n,y) & z+x=y":
def diff5 "?msd_fibb5 Ex,y $min5(n,x) & $fac5(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):113 states - 0ms
 (z+x)=y:16 states - 0ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):106 states - 1ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):35 states - 0ms
Total computation time: 2ms.

(min1(n,x))&fac1(i,n,y))):86 states - 0ms
 (z+x)=y:16 states - 0ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):81 states - 1ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):26 states - 0ms
Total computation time: 1ms.

(min2(n,x))&fac2(i,n,y))):202 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):191 states - 1ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):54 states - 0ms
Total computation time: 2ms.

(min3(n,x))&fac3(i,n,y))):178 states - 0ms
 (z+x)=y:16 states - 0ms
  ((min3(n,x))&fac3(i,n,y)))&(z+x)=y):166 states - 1ms
   (E x , y ((min3(n,x))&fac3(i,n,y)))&(z+x)=y)):41 states - 0ms
Total computation time: 2ms.

(min4(n,x))&fac4(i,n,y))):134 states - 0ms
 (z+x)=y:16 states - 1ms
  ((min4(n,x))&fac4(i,n,y)))&(z+x)=y):130 states - 0ms
   (E x

In [46]:
def abeq_fibb5 "?msd_fibb5 $bordercond5(i,j,5,n+4) & (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z)) & (Ez $diff3(i,n,z) &  $diff3(j,n,z)) & (Ez $diff4(i,n,z) &  $diff4(j,n,z)) & (Ez $diff5(i,n,z) &  $diff5(j,n,z))":
def abeq_fib5 "?msd_fib (n<4 & $feq_fib(i,j,n)) | (n>=4 & (?msd_fibb5 Eii,jj,nn ($conv_fib_fibb5(?msd_fib i, ?msd_fibb5 ii) & $conv_fib_fibb5(?msd_fib j, ?msd_fibb5 jj) & $conv_fib_fibb5(?msd_fib n, ?msd_fibb5 nn) & $abeq_fibb5(ii,jj,nn-4))))":

computed ~:1 states - 1ms
computed ~:2 states - 0ms
(diff0(i,n,z))&diff0(j,n,z))):195 states - 1ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):201 states - 3ms
  (bordercond5(i,j,5,(n+4)))&(E z (diff0(i,n,z))&diff0(j,n,z))))):97 states - 1ms
   (diff1(i,n,z))&diff1(j,n,z))):106 states - 1ms
    (E z (diff1(i,n,z))&diff1(j,n,z)))):102 states - 1ms
     ((bordercond5(i,j,5,(n+4)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):87 states - 0ms
      (diff2(i,n,z))&diff2(j,n,z))):327 states - 3ms
       (E z (diff2(i,n,z))&diff2(j,n,z)))):285 states - 2ms
        (((bordercond5(i,j,5,(n+4)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):95 states - 1ms
         (diff3(i,n,z))&diff3(j,n,z))):210 states - 2ms
          (E z (diff3(i,n,z))&diff3(j,n,z)))):195 states - 1ms
           ((((bordercond5(i,j,5,(n+4)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff

In [47]:
eval comp_fib5 n "?msd_fib Aj $abeq_fib5(i,j,n) => i<=j":

i<=j:6 states - 0ms
 (abeq_fib5(i,j,n))=>i<=j):126 states - 2ms
  (A j (abeq_fib5(i,j,n))=>i<=j)):28 states - 2ms
Total computation time: 4ms.


### Apply the semigroup trick

In [48]:
%SGT comp_fib5 msd_fib Comp_fib5

Reading Result/comp_fib5.mpl
Rank: 8
12 states
values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Writing Word Automata Library/Comp_fib5.txt
Done.

## 6-abelian complexity

Define the 6-block map of fib and construct the conversion predicate
between both numeration systems.

In [49]:
%%python
s6 = block(s, 6)
ns6 = address(s6, "fibb6")
ns6.gen_ns()
(ns-ns6).gen_dfa("conv_fib_fibb6")

>>> Address automaton for block(subst(0->01, 1->0), 6)
  DT polynomial: X^2-X-1 (n=2)
  θ=1.618033988749895
  DT automaton: 7 states, 7 final states, 11 transitions.

>>> address(block(subst(0->01, 1->0), 6), 'fibb6') is generating it's NS
  Numeration system automaton: 2 states, 2 final states, 3 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.ftvsMYWcSS/Walnut/Custom Bases/msd_fibb6.txt in format Walnut
  +address(block(subst(0->01, 1->0), 6), 'fibb6')+address(block(subst(0->01, 1->0), 6), 'fibb6')-address(block(subst(0->01, 1->0), 6), 'fibb6') is generating it's dfa under name msd_fibb6_addition
  Combination polynomial is X^2-X-1
  θ=1.618033988749895
  Combination automaton: 343 states, 343 final states, 1331 transitions.
  (P,u) vector bound: (0.0, 0.0) + (3.5124611797498106, 4.959674775249768)*C
  Fast poly bound: C in [(-1, -1), (2, 2)] => (14.049844718999243, 19.838699100999072)
Expanding DFA...done.
  Flattened automaton : 3206 states, 126 final st

Translate the border condition predicate into the current numeration
system.

In [50]:
def bordercond6 "?msd_fibb6 (?msd_fib Eii,jj,kk,nn 
($conv_fib_fibb6(?msd_fib ii, ?msd_fibb6 i) & 
$conv_fib_fibb6(?msd_fib jj, ?msd_fibb6 j) & 
$conv_fib_fibb6(?msd_fib kk, ?msd_fibb6 k) & 
$conv_fib_fibb6(?msd_fib nn, ?msd_fibb6 n) & 
$bordercond(ii,jj,kk,nn)))":






(conv_fib_fibb6(ii,i))&conv_fib_fibb6(jj,j))):4 states - 0ms
 ((conv_fib_fibb6(ii,i))&conv_fib_fibb6(jj,j)))&conv_fib_fibb6(kk,k))):8 states - 0ms
  (((conv_fib_fibb6(ii,i))&conv_fib_fibb6(jj,j)))&conv_fib_fibb6(kk,k)))&conv_fib_fibb6(nn,n))):16 states - 0ms
   ((((conv_fib_fibb6(ii,i))&conv_fib_fibb6(jj,j)))&conv_fib_fibb6(kk,k)))&conv_fib_fibb6(nn,n)))&bordercond(ii,jj,kk,nn))):44 states - 1ms
    (E ii , jj , kk , nn ((((conv_fib_fibb6(ii,i))&conv_fib_fibb6(jj,j)))&conv_fib_fibb6(kk,k)))&conv_fib_fibb6(nn,n)))&bordercond(ii,jj,kk,nn)))):44 states - 1ms
Total computation time: 2ms.


### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Fibb6.

In [51]:
%%python
for (i,a) in enumerate(ns6.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s6, ns6.ns, **w)
    (parikh - ns6).gen_dfa(f"fibb6p{i}")

>>> Address automaton for block(subst(0->01, 1->0), 6)
  DT polynomial: X^4-X^3-X^2 (n=4)
  θ=1.618033988749895 (X^2)
  DT automaton: 7 states, 7 final states, 11 transitions.

>>> +address(block(subst(0->01, 1->0), 6), 'fibb6', '_': 0, 'a': 1)-address(block(subst(0->01, 1->0), 6), 'fibb6') is generating it's dfa under name fibb6p0
  Combination polynomial is X^4-X^3-X^2
  θ=1.618033988749895 (X^2)
  Combination automaton: 49 states, 49 final states, 121 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0, 0.0) + (10.577708763999668, 13.984597134749396, 19.49705831449921, 28.416407864998753)*C
  Fast poly bound: C in [(-1, -1, -1, -1), (0, 0, 1, 1)] => (42.31083505599867, 55.938388538997586, 77.98823325799684, 113.66563145999501)
Expanding DFA...done.
  Flattened automaton : 1915 states, 33 final states, 2965 transitions.
  Trimmed flattened automaton : 58 states, 33 final states, 90 transitions.
  Real bound is [1, 2, 3, 4] (vs [42.31083505599867, 55.938388538997586, 77.98823325799684, 

### Compute the complexity function

Use Walnut to generate a linear representation for the 6-abelian
complexity.

In [52]:
def fac0 "?msd_fibb6 Ex,y $fibb6p0(i,x) & $fibb6p0(i+n,y) & z+x=y":
def fac1 "?msd_fibb6 Ex,y $fibb6p1(i,x) & $fibb6p1(i+n,y) & z+x=y":
def fac2 "?msd_fibb6 Ex,y $fibb6p2(i,x) & $fibb6p2(i+n,y) & z+x=y":
def fac3 "?msd_fibb6 Ex,y $fibb6p3(i,x) & $fibb6p3(i+n,y) & z+x=y":
def fac4 "?msd_fibb6 Ex,y $fibb6p4(i,x) & $fibb6p4(i+n,y) & z+x=y":
def fac5 "?msd_fibb6 Ex,y $fibb6p5(i,x) & $fibb6p5(i+n,y) & z+x=y":
def fac6 "?msd_fibb6 Ex,y $fibb6p6(i,x) & $fibb6p6(i+n,y) & z+x=y":

(fibb6p0(i,x))&fibb6p0((i+n),y))):164 states - 0ms
 (z+x)=y:16 states - 1ms
  ((fibb6p0(i,x))&fibb6p0((i+n),y)))&(z+x)=y):330 states - 1ms
   (E x , y ((fibb6p0(i,x))&fibb6p0((i+n),y)))&(z+x)=y)):93 states - 1ms
Total computation time: 3ms.

(fibb6p1(i,x))&fibb6p1((i+n),y))):315 states - 1ms
 (z+x)=y:16 states - 0ms
  ((fibb6p1(i,x))&fibb6p1((i+n),y)))&(z+x)=y):672 states - 5ms
   (E x , y ((fibb6p1(i,x))&fibb6p1((i+n),y)))&(z+x)=y)):191 states - 1ms
Total computation time: 8ms.

(fibb6p2(i,x))&fibb6p2((i+n),y))):282 states - 1ms
 (z+x)=y:16 states - 0ms
  ((fibb6p2(i,x))&fibb6p2((i+n),y)))&(z+x)=y):626 states - 6ms
   (E x , y ((fibb6p2(i,x))&fibb6p2((i+n),y)))&(z+x)=y)):168 states - 1ms
Total computation time: 9ms.

(fibb6p3(i,x))&fibb6p3((i+n),y))):252 states - 1ms
 (z+x)=y:16 states - 0ms
  ((fibb6p3(i,x))&fibb6p3((i+n),y)))&(z+x)=y):602 states - 9ms
   (E x , y ((fibb6p3(i,x))&fibb6p3((i+n),y)))&(z+x)=y)):142 states - 1ms
Total computation time: 14ms.

(fibb6p4(i,x))&fibb6p4((i+n)

In [53]:
def min0 "?msd_fibb6 Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_fibb6 Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_fibb6 Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":
def min3 "?msd_fibb6 Ei $fac3(i,n,x) & Aj,y $fac3(j,n,y) => y>=x":
def min4 "?msd_fibb6 Ei $fac4(i,n,x) & Aj,y $fac4(j,n,y) => y>=x":
def min5 "?msd_fibb6 Ei $fac5(i,n,x) & Aj,y $fac5(j,n,y) => y>=x":
def min6 "?msd_fibb6 Ei $fac6(i,n,x) & Aj,y $fac6(j,n,y) => y>=x":

y>=x:6 states - 0ms
 (fac0(j,n,y))=>y>=x):339 states - 8ms
  (A j , y (fac0(j,n,y))=>y>=x)):12 states - 8ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):59 states - 0ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):11 states - 0ms
Total computation time: 17ms.

y>=x:6 states - 0ms
 (fac1(j,n,y))=>y>=x):646 states - 22ms
  (A j , y (fac1(j,n,y))=>y>=x)):17 states - 17ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):121 states - 0ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):17 states - 1ms
Total computation time: 41ms.

y>=x:6 states - 0ms
 (fac2(j,n,y))=>y>=x):566 states - 17ms
  (A j , y (fac2(j,n,y))=>y>=x)):17 states - 13ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):101 states - 0ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):17 states - 1ms
Total computation time: 32ms.

y>=x:6 states - 0ms
 (fac3(j,n,y))=>y>=x):503 states - 12ms
  (A j , y (fac3(j,n,y))=>y>=x)):17 states - 11ms
   (fac3(i,n,x))&(A j , y (fac3(j,n,y))=>y>=x))):84 stat

In [54]:
def diff0 "?msd_fibb6 Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_fibb6 Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_fibb6 Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":
def diff3 "?msd_fibb6 Ex,y $min3(n,x) & $fac3(i,n,y) & z+x=y":
def diff4 "?msd_fibb6 Ex,y $min4(n,x) & $fac4(i,n,y) & z+x=y":
def diff5 "?msd_fibb6 Ex,y $min5(n,x) & $fac5(i,n,y) & z+x=y":
def diff6 "?msd_fibb6 Ex,y $min6(n,x) & $fac6(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):113 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):106 states - 0ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):35 states - 1ms
Total computation time: 2ms.

(min1(n,x))&fac1(i,n,y))):229 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):214 states - 1ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):62 states - 1ms
Total computation time: 3ms.

(min2(n,x))&fac2(i,n,y))):202 states - 0ms
 (z+x)=y:16 states - 0ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):191 states - 1ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):54 states - 1ms
Total computation time: 3ms.

(min3(n,x))&fac3(i,n,y))):178 states - 1ms
 (z+x)=y:16 states - 1ms
  ((min3(n,x))&fac3(i,n,y)))&(z+x)=y):166 states - 5ms
   (E x , y ((min3(n,x))&fac3(i,n,y)))&(z+x)=y)):41 states - 2ms
Total computation time: 10ms.

(min4(n,x))&fac4(i,n,y))):134 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min4(n,x))&fac4(i,n,y)))&(z+x)=y):130 states - 4ms
   (

In [55]:
def abeq_fibb6 "?msd_fibb6 $bordercond6(i,j,6,n+5) & (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z)) & (Ez $diff3(i,n,z) &  $diff3(j,n,z)) & (Ez $diff4(i,n,z) &  $diff4(j,n,z)) & (Ez $diff5(i,n,z) &  $diff5(j,n,z)) & (Ez $diff6(i,n,z) &  $diff6(j,n,z))":
def abeq_fib6 "?msd_fib (n<5 & $feq_fib(i,j,n)) | (n>=5 & (?msd_fibb6 Eii,jj,nn ($conv_fib_fibb6(?msd_fib i, ?msd_fibb6 ii) & $conv_fib_fibb6(?msd_fib j, ?msd_fibb6 jj) & $conv_fib_fibb6(?msd_fib n, ?msd_fibb6 nn) & $abeq_fibb6(ii,jj,nn-5))))":

computed ~:1 states - 1ms
computed ~:2 states - 0ms
(diff0(i,n,z))&diff0(j,n,z))):195 states - 1ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):201 states - 1ms
  (bordercond6(i,j,6,(n+5)))&(E z (diff0(i,n,z))&diff0(j,n,z))))):93 states - 1ms
   (diff1(i,n,z))&diff1(j,n,z))):455 states - 3ms
    (E z (diff1(i,n,z))&diff1(j,n,z)))):431 states - 2ms
     ((bordercond6(i,j,6,(n+5)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):115 states - 1ms
      (diff2(i,n,z))&diff2(j,n,z))):327 states - 1ms
       (E z (diff2(i,n,z))&diff2(j,n,z)))):285 states - 2ms
        (((bordercond6(i,j,6,(n+5)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):115 states - 1ms
         (diff3(i,n,z))&diff3(j,n,z))):210 states - 1ms
          (E z (diff3(i,n,z))&diff3(j,n,z)))):195 states - 1ms
           ((((bordercond6(i,j,6,(n+5)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&di

In [56]:
eval comp_fib6 n "?msd_fib Aj $abeq_fib6(i,j,n) => i<=j":

i<=j:6 states - 0ms
 (abeq_fib6(i,j,n))=>i<=j):137 states - 2ms
  (A j (abeq_fib6(i,j,n))=>i<=j)):33 states - 2ms
Total computation time: 5ms.


### Apply the semigroup trick

In [57]:
%SGT comp_fib6 msd_fib Comp_fib6

Reading Result/comp_fib6.mpl
Rank: 9
14 states
values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Writing Word Automata Library/Comp_fib6.txt
Done.

## 7-abelian complexity

Define the 7-block map of fib and construct the conversion predicate
between both numeration systems.

In [58]:
%%python
s7 = block(s, 7)
ns7 = address(s7, "fibb7")
ns7.gen_ns()
(ns-ns7).gen_dfa("conv_fib_fibb7")

>>> Address automaton for block(subst(0->01, 1->0), 7)
  DT polynomial: X^2-X-1 (n=2)
  θ=1.618033988749895
  DT automaton: 8 states, 8 final states, 13 transitions.

>>> address(block(subst(0->01, 1->0), 7), 'fibb7') is generating it's NS
  Numeration system automaton: 2 states, 2 final states, 3 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.ftvsMYWcSS/Walnut/Custom Bases/msd_fibb7.txt in format Walnut
  +address(block(subst(0->01, 1->0), 7), 'fibb7')+address(block(subst(0->01, 1->0), 7), 'fibb7')-address(block(subst(0->01, 1->0), 7), 'fibb7') is generating it's dfa under name msd_fibb7_addition
  Combination polynomial is X^2-X-1
  θ=1.618033988749895
  Combination automaton: 512 states, 512 final states, 2197 transitions.
  (P,u) vector bound: (0.0, 0.0) + (3.5124611797498106, 4.959674775249768)*C
  Fast poly bound: C in [(-1, -1), (2, 2)] => (14.049844718999243, 19.838699100999072)
Expanding DFA...done.
  Flattened automaton : 4137 states, 164 final st

Translate the border condition predicate into the current numeration
system.

In [59]:
def bordercond7 "?msd_fibb7 (?msd_fib Eii,jj,kk,nn 
($conv_fib_fibb7(?msd_fib ii, ?msd_fibb7 i) & 
$conv_fib_fibb7(?msd_fib jj, ?msd_fibb7 j) & 
$conv_fib_fibb7(?msd_fib kk, ?msd_fibb7 k) & 
$conv_fib_fibb7(?msd_fib nn, ?msd_fibb7 n) & 
$bordercond(ii,jj,kk,nn)))":






(conv_fib_fibb7(ii,i))&conv_fib_fibb7(jj,j))):4 states - 0ms
 ((conv_fib_fibb7(ii,i))&conv_fib_fibb7(jj,j)))&conv_fib_fibb7(kk,k))):8 states - 0ms
  (((conv_fib_fibb7(ii,i))&conv_fib_fibb7(jj,j)))&conv_fib_fibb7(kk,k)))&conv_fib_fibb7(nn,n))):16 states - 0ms
   ((((conv_fib_fibb7(ii,i))&conv_fib_fibb7(jj,j)))&conv_fib_fibb7(kk,k)))&conv_fib_fibb7(nn,n)))&bordercond(ii,jj,kk,nn))):44 states - 1ms
    (E ii , jj , kk , nn ((((conv_fib_fibb7(ii,i))&conv_fib_fibb7(jj,j)))&conv_fib_fibb7(kk,k)))&conv_fib_fibb7(nn,n)))&bordercond(ii,jj,kk,nn)))):44 states - 1ms
Total computation time: 2ms.


### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Fibb7.

In [60]:
%%python
for (i,a) in enumerate(ns7.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s7, ns7.ns, **w)
    (parikh - ns7).gen_dfa(f"fibb7p{i}")

>>> Address automaton for block(subst(0->01, 1->0), 7)
  DT polynomial: X^5-X^4-X^3 (n=5)
  θ=1.618033988749895 (X^3)
  DT automaton: 8 states, 8 final states, 13 transitions.

>>> +address(block(subst(0->01, 1->0), 7), 'fibb7', '_': 0, 'a': 1)-address(block(subst(0->01, 1->0), 7), 'fibb7') is generating it's dfa under name fibb7p0
  Combination polynomial is X^5-X^4-X^3
  θ=1.618033988749895 (X^3)
  Combination automaton: 64 states, 64 final states, 169 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0, 0.0, 0.0) + (18.195742752749567, 23.90394668524894, 33.14001466274874, 48.08428657274791, 72.26462646024687)*C
  Fast poly bound: C in [(-1, -1, -1, -1, -1), (0, 0, 1, 1, 1)] => (90.97871376374783, 119.5197334262447, 165.7000733137437, 240.42143286373954, 361.32313230123435)
Expanding DFA...done.
  Flattened automaton : 6564 states, 59 final states, 10601 transitions.
  Trimmed flattened automaton : 110 states, 59 final states, 177 transitions.
  Real bound is [1, 2, 3, 4, 7] (vs [90.9

### Compute the complexity function

Use Walnut to generate a linear representation for the 7-abelian
complexity.

In [61]:
def fac0 "?msd_fibb7 Ex,y $fibb7p0(i,x) & $fibb7p0(i+n,y) & z+x=y":
def fac1 "?msd_fibb7 Ex,y $fibb7p1(i,x) & $fibb7p1(i+n,y) & z+x=y":
def fac2 "?msd_fibb7 Ex,y $fibb7p2(i,x) & $fibb7p2(i+n,y) & z+x=y":
def fac3 "?msd_fibb7 Ex,y $fibb7p3(i,x) & $fibb7p3(i+n,y) & z+x=y":
def fac4 "?msd_fibb7 Ex,y $fibb7p4(i,x) & $fibb7p4(i+n,y) & z+x=y":
def fac5 "?msd_fibb7 Ex,y $fibb7p5(i,x) & $fibb7p5(i+n,y) & z+x=y":
def fac6 "?msd_fibb7 Ex,y $fibb7p6(i,x) & $fibb7p6(i+n,y) & z+x=y":
def fac7 "?msd_fibb7 Ex,y $fibb7p7(i,x) & $fibb7p7(i+n,y) & z+x=y":

(fibb7p0(i,x))&fibb7p0((i+n),y))):357 states - 1ms
 (z+x)=y:16 states - 0ms
  ((fibb7p0(i,x))&fibb7p0((i+n),y)))&(z+x)=y):759 states - 5ms
   (E x , y ((fibb7p0(i,x))&fibb7p0((i+n),y)))&(z+x)=y)):239 states - 2ms
Total computation time: 9ms.

(fibb7p1(i,x))&fibb7p1((i+n),y))):315 states - 1ms
 (z+x)=y:16 states - 0ms
  ((fibb7p1(i,x))&fibb7p1((i+n),y)))&(z+x)=y):672 states - 5ms
   (E x , y ((fibb7p1(i,x))&fibb7p1((i+n),y)))&(z+x)=y)):191 states - 2ms
Total computation time: 9ms.

(fibb7p2(i,x))&fibb7p2((i+n),y))):282 states - 1ms
 (z+x)=y:16 states - 0ms
  ((fibb7p2(i,x))&fibb7p2((i+n),y)))&(z+x)=y):626 states - 5ms
   (E x , y ((fibb7p2(i,x))&fibb7p2((i+n),y)))&(z+x)=y)):168 states - 1ms
Total computation time: 8ms.

(fibb7p3(i,x))&fibb7p3((i+n),y))):252 states - 1ms
 (z+x)=y:16 states - 0ms
  ((fibb7p3(i,x))&fibb7p3((i+n),y)))&(z+x)=y):602 states - 4ms
   (E x , y ((fibb7p3(i,x))&fibb7p3((i+n),y)))&(z+x)=y)):142 states - 1ms
Total computation time: 8ms.

(fibb7p4(i,x))&fibb7p4((i+n)

In [62]:
def min0 "?msd_fibb7 Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_fibb7 Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_fibb7 Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":
def min3 "?msd_fibb7 Ei $fac3(i,n,x) & Aj,y $fac3(j,n,y) => y>=x":
def min4 "?msd_fibb7 Ei $fac4(i,n,x) & Aj,y $fac4(j,n,y) => y>=x":
def min5 "?msd_fibb7 Ei $fac5(i,n,x) & Aj,y $fac5(j,n,y) => y>=x":
def min6 "?msd_fibb7 Ei $fac6(i,n,x) & Aj,y $fac6(j,n,y) => y>=x":
def min7 "?msd_fibb7 Ei $fac7(i,n,x) & Aj,y $fac7(j,n,y) => y>=x":

y>=x:6 states - 0ms
 (fac0(j,n,y))=>y>=x):802 states - 25ms
  (A j , y (fac0(j,n,y))=>y>=x)):17 states - 23ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):145 states - 1ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):17 states - 0ms
Total computation time: 50ms.

y>=x:6 states - 0ms
 (fac1(j,n,y))=>y>=x):646 states - 39ms
  (A j , y (fac1(j,n,y))=>y>=x)):17 states - 39ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):121 states - 0ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):17 states - 1ms
Total computation time: 80ms.

y>=x:6 states - 1ms
 (fac2(j,n,y))=>y>=x):566 states - 16ms
  (A j , y (fac2(j,n,y))=>y>=x)):17 states - 12ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):101 states - 0ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):17 states - 0ms
Total computation time: 30ms.

y>=x:6 states - 0ms
 (fac3(j,n,y))=>y>=x):503 states - 17ms
  (A j , y (fac3(j,n,y))=>y>=x)):17 states - 11ms
   (fac3(i,n,x))&(A j , y (fac3(j,n,y))=>y>=x))):84 s

In [63]:
def diff0 "?msd_fibb7 Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_fibb7 Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_fibb7 Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":
def diff3 "?msd_fibb7 Ex,y $min3(n,x) & $fac3(i,n,y) & z+x=y":
def diff4 "?msd_fibb7 Ex,y $min4(n,x) & $fac4(i,n,y) & z+x=y":
def diff5 "?msd_fibb7 Ex,y $min5(n,x) & $fac5(i,n,y) & z+x=y":
def diff6 "?msd_fibb7 Ex,y $min6(n,x) & $fac6(i,n,y) & z+x=y":
def diff7 "?msd_fibb7 Ex,y $min7(n,x) & $fac7(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):280 states - 0ms
 (z+x)=y:16 states - 1ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):265 states - 1ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):82 states - 1ms
Total computation time: 4ms.

(min1(n,x))&fac1(i,n,y))):229 states - 0ms
 (z+x)=y:16 states - 1ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):214 states - 1ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):62 states - 0ms
Total computation time: 3ms.

(min2(n,x))&fac2(i,n,y))):202 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):191 states - 1ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):54 states - 0ms
Total computation time: 2ms.

(min3(n,x))&fac3(i,n,y))):178 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min3(n,x))&fac3(i,n,y)))&(z+x)=y):166 states - 2ms
   (E x , y ((min3(n,x))&fac3(i,n,y)))&(z+x)=y)):41 states - 1ms
Total computation time: 5ms.

(min4(n,x))&fac4(i,n,y))):134 states - 0ms
 (z+x)=y:16 states - 0ms
  ((min4(n,x))&fac4(i,n,y)))&(z+x)=y):130 states - 1ms
   (E

In [64]:
def abeq_fibb7 "?msd_fibb7 $bordercond7(i,j,7,n+6) & (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z)) & (Ez $diff3(i,n,z) &  $diff3(j,n,z)) & (Ez $diff4(i,n,z) &  $diff4(j,n,z)) & (Ez $diff5(i,n,z) &  $diff5(j,n,z)) & (Ez $diff6(i,n,z) &  $diff6(j,n,z)) & (Ez $diff7(i,n,z) &  $diff7(j,n,z))":
def abeq_fib7 "?msd_fib (n<6 & $feq_fib(i,j,n)) | (n>=6 & (?msd_fibb7 Eii,jj,nn ($conv_fib_fibb7(?msd_fib i, ?msd_fibb7 ii) & $conv_fib_fibb7(?msd_fib j, ?msd_fibb7 jj) & $conv_fib_fibb7(?msd_fib n, ?msd_fibb7 nn) & $abeq_fibb7(ii,jj,nn-6))))":

computed ~:1 states - 1ms
computed ~:2 states - 0ms
(diff0(i,n,z))&diff0(j,n,z))):775 states - 5ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):766 states - 5ms
  (bordercond7(i,j,7,(n+6)))&(E z (diff0(i,n,z))&diff0(j,n,z))))):151 states - 2ms
   (diff1(i,n,z))&diff1(j,n,z))):455 states - 3ms
    (E z (diff1(i,n,z))&diff1(j,n,z)))):431 states - 2ms
     ((bordercond7(i,j,7,(n+6)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):141 states - 2ms
      (diff2(i,n,z))&diff2(j,n,z))):327 states - 2ms
       (E z (diff2(i,n,z))&diff2(j,n,z)))):285 states - 2ms
        (((bordercond7(i,j,7,(n+6)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):134 states - 1ms
         (diff3(i,n,z))&diff3(j,n,z))):210 states - 1ms
          (E z (diff3(i,n,z))&diff3(j,n,z)))):195 states - 1ms
           ((((bordercond7(i,j,7,(n+6)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&d

In [65]:
eval comp_fib7 n "?msd_fib Aj $abeq_fib7(i,j,n) => i<=j":

i<=j:6 states - 0ms
 (abeq_fib7(i,j,n))=>i<=j):173 states - 3ms
  (A j (abeq_fib7(i,j,n))=>i<=j)):44 states - 3ms
Total computation time: 7ms.


### Apply the semigroup trick

In [66]:
%SGT comp_fib7 msd_fib Comp_fib7

Reading Result/comp_fib7.mpl
Rank: 10
16 states
values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Writing Word Automata Library/Comp_fib7.txt
Done.

## 8-abelian complexity

Define the 8-block map of fib and construct the conversion predicate
between both numeration systems.

In [67]:
%%python
s8 = block(s, 8)
ns8 = address(s8, "fibb8")
ns8.gen_ns()
(ns-ns8).gen_dfa("conv_fib_fibb8")

>>> Address automaton for block(subst(0->01, 1->0), 8)
  DT polynomial: X^2-X-1 (n=2)
  θ=1.618033988749895
  DT automaton: 9 states, 9 final states, 14 transitions.

>>> address(block(subst(0->01, 1->0), 8), 'fibb8') is generating it's NS
  Numeration system automaton: 2 states, 2 final states, 3 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.ftvsMYWcSS/Walnut/Custom Bases/msd_fibb8.txt in format Walnut
  +address(block(subst(0->01, 1->0), 8), 'fibb8')+address(block(subst(0->01, 1->0), 8), 'fibb8')-address(block(subst(0->01, 1->0), 8), 'fibb8') is generating it's dfa under name msd_fibb8_addition
  Combination polynomial is X^2-X-1
  θ=1.618033988749895
  Combination automaton: 729 states, 729 final states, 2744 transitions.
  (P,u) vector bound: (0.0, 0.0) + (3.5124611797498106, 4.959674775249768)*C
  Fast poly bound: C in [(-1, -1), (2, 2)] => (14.049844718999243, 19.838699100999072)
Expanding DFA...done.
  Flattened automaton : 5173 states, 207 final st

Translate the border condition predicate into the current numeration
system.

In [68]:
def bordercond8 "?msd_fibb8 (?msd_fib Eii,jj,kk,nn 
($conv_fib_fibb8(?msd_fib ii, ?msd_fibb8 i) & 
$conv_fib_fibb8(?msd_fib jj, ?msd_fibb8 j) & 
$conv_fib_fibb8(?msd_fib kk, ?msd_fibb8 k) & 
$conv_fib_fibb8(?msd_fib nn, ?msd_fibb8 n) & 
$bordercond(ii,jj,kk,nn)))":






(conv_fib_fibb8(ii,i))&conv_fib_fibb8(jj,j))):4 states - 0ms
 ((conv_fib_fibb8(ii,i))&conv_fib_fibb8(jj,j)))&conv_fib_fibb8(kk,k))):8 states - 0ms
  (((conv_fib_fibb8(ii,i))&conv_fib_fibb8(jj,j)))&conv_fib_fibb8(kk,k)))&conv_fib_fibb8(nn,n))):16 states - 0ms
   ((((conv_fib_fibb8(ii,i))&conv_fib_fibb8(jj,j)))&conv_fib_fibb8(kk,k)))&conv_fib_fibb8(nn,n)))&bordercond(ii,jj,kk,nn))):44 states - 1ms
    (E ii , jj , kk , nn ((((conv_fib_fibb8(ii,i))&conv_fib_fibb8(jj,j)))&conv_fib_fibb8(kk,k)))&conv_fib_fibb8(nn,n)))&bordercond(ii,jj,kk,nn)))):44 states - 0ms
Total computation time: 2ms.


### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Fibb8.

In [69]:
%%python
for (i,a) in enumerate(ns8.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s8, ns8.ns, **w)
    (parikh - ns8).gen_dfa(f"fibb8p{i}")

>>> Address automaton for block(subst(0->01, 1->0), 8)
  DT polynomial: X^5-X^4-X^3 (n=5)
  θ=1.618033988749895 (X^3)
  DT automaton: 9 states, 9 final states, 14 transitions.

>>> +address(block(subst(0->01, 1->0), 8), 'fibb8', '_': 0, 'a': 1)-address(block(subst(0->01, 1->0), 8), 'fibb8') is generating it's dfa under name fibb8p0
  Combination polynomial is X^5-X^4-X^3
  θ=1.618033988749895 (X^3)
  Combination automaton: 81 states, 81 final states, 196 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0, 0.0, 0.0) + (18.195742752749567, 23.90394668524894, 33.14001466274874, 48.08428657274791, 72.26462646024687)*C
  Fast poly bound: C in [(-1, -1, -1, -1, -1), (0, 0, 1, 1, 1)] => (90.97871376374783, 119.5197334262447, 165.7000733137437, 240.42143286373954, 361.32313230123435)
Expanding DFA...done.
  Flattened automaton : 7376 states, 67 final states, 11402 transitions.
  Trimmed flattened automaton : 122 states, 67 final states, 190 transitions.
  Real bound is [1, 2, 3, 4, 7] (vs [90.9

### Compute the complexity function

Use Walnut to generate a linear representation for the 8-abelian
complexity.

In [70]:
def fac0 "?msd_fibb8 Ex,y $fibb8p0(i,x) & $fibb8p0(i+n,y) & z+x=y":
def fac1 "?msd_fibb8 Ex,y $fibb8p1(i,x) & $fibb8p1(i+n,y) & z+x=y":
def fac2 "?msd_fibb8 Ex,y $fibb8p2(i,x) & $fibb8p2(i+n,y) & z+x=y":
def fac3 "?msd_fibb8 Ex,y $fibb8p3(i,x) & $fibb8p3(i+n,y) & z+x=y":
def fac4 "?msd_fibb8 Ex,y $fibb8p4(i,x) & $fibb8p4(i+n,y) & z+x=y":
def fac5 "?msd_fibb8 Ex,y $fibb8p5(i,x) & $fibb8p5(i+n,y) & z+x=y":
def fac6 "?msd_fibb8 Ex,y $fibb8p6(i,x) & $fibb8p6(i+n,y) & z+x=y":
def fac7 "?msd_fibb8 Ex,y $fibb8p7(i,x) & $fibb8p7(i+n,y) & z+x=y":
def fac8 "?msd_fibb8 Ex,y $fibb8p8(i,x) & $fibb8p8(i+n,y) & z+x=y":

(fibb8p0(i,x))&fibb8p0((i+n),y))):357 states - 1ms
 (z+x)=y:16 states - 0ms
  ((fibb8p0(i,x))&fibb8p0((i+n),y)))&(z+x)=y):759 states - 5ms
   (E x , y ((fibb8p0(i,x))&fibb8p0((i+n),y)))&(z+x)=y)):239 states - 1ms
Total computation time: 8ms.

(fibb8p1(i,x))&fibb8p1((i+n),y))):315 states - 1ms
 (z+x)=y:16 states - 0ms
  ((fibb8p1(i,x))&fibb8p1((i+n),y)))&(z+x)=y):672 states - 5ms
   (E x , y ((fibb8p1(i,x))&fibb8p1((i+n),y)))&(z+x)=y)):191 states - 1ms
Total computation time: 8ms.

(fibb8p2(i,x))&fibb8p2((i+n),y))):282 states - 1ms
 (z+x)=y:16 states - 0ms
  ((fibb8p2(i,x))&fibb8p2((i+n),y)))&(z+x)=y):626 states - 4ms
   (E x , y ((fibb8p2(i,x))&fibb8p2((i+n),y)))&(z+x)=y)):168 states - 1ms
Total computation time: 6ms.

(fibb8p3(i,x))&fibb8p3((i+n),y))):252 states - 0ms
 (z+x)=y:16 states - 0ms
  ((fibb8p3(i,x))&fibb8p3((i+n),y)))&(z+x)=y):602 states - 3ms
   (E x , y ((fibb8p3(i,x))&fibb8p3((i+n),y)))&(z+x)=y)):142 states - 1ms
Total computation time: 5ms.

(fibb8p4(i,x))&fibb8p4((i+n)

In [71]:
def min0 "?msd_fibb8 Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_fibb8 Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_fibb8 Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":
def min3 "?msd_fibb8 Ei $fac3(i,n,x) & Aj,y $fac3(j,n,y) => y>=x":
def min4 "?msd_fibb8 Ei $fac4(i,n,x) & Aj,y $fac4(j,n,y) => y>=x":
def min5 "?msd_fibb8 Ei $fac5(i,n,x) & Aj,y $fac5(j,n,y) => y>=x":
def min6 "?msd_fibb8 Ei $fac6(i,n,x) & Aj,y $fac6(j,n,y) => y>=x":
def min7 "?msd_fibb8 Ei $fac7(i,n,x) & Aj,y $fac7(j,n,y) => y>=x":
def min8 "?msd_fibb8 Ei $fac8(i,n,x) & Aj,y $fac8(j,n,y) => y>=x":

y>=x:6 states - 0ms
 (fac0(j,n,y))=>y>=x):802 states - 24ms
  (A j , y (fac0(j,n,y))=>y>=x)):17 states - 17ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):145 states - 0ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):17 states - 0ms
Total computation time: 43ms.

y>=x:6 states - 0ms
 (fac1(j,n,y))=>y>=x):646 states - 18ms
  (A j , y (fac1(j,n,y))=>y>=x)):17 states - 14ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):121 states - 0ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):17 states - 0ms
Total computation time: 33ms.

y>=x:6 states - 0ms
 (fac2(j,n,y))=>y>=x):566 states - 16ms
  (A j , y (fac2(j,n,y))=>y>=x)):17 states - 12ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):101 states - 0ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):17 states - 1ms
Total computation time: 30ms.

y>=x:6 states - 0ms
 (fac3(j,n,y))=>y>=x):503 states - 14ms
  (A j , y (fac3(j,n,y))=>y>=x)):17 states - 11ms
   (fac3(i,n,x))&(A j , y (fac3(j,n,y))=>y>=x))):84 s

In [72]:
def diff0 "?msd_fibb8 Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_fibb8 Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_fibb8 Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":
def diff3 "?msd_fibb8 Ex,y $min3(n,x) & $fac3(i,n,y) & z+x=y":
def diff4 "?msd_fibb8 Ex,y $min4(n,x) & $fac4(i,n,y) & z+x=y":
def diff5 "?msd_fibb8 Ex,y $min5(n,x) & $fac5(i,n,y) & z+x=y":
def diff6 "?msd_fibb8 Ex,y $min6(n,x) & $fac6(i,n,y) & z+x=y":
def diff7 "?msd_fibb8 Ex,y $min7(n,x) & $fac7(i,n,y) & z+x=y":
def diff8 "?msd_fibb8 Ex,y $min8(n,x) & $fac8(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):280 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):265 states - 3ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):82 states - 0ms
Total computation time: 5ms.

(min1(n,x))&fac1(i,n,y))):229 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):214 states - 2ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):62 states - 0ms
Total computation time: 3ms.

(min2(n,x))&fac2(i,n,y))):202 states - 0ms
 (z+x)=y:16 states - 1ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):191 states - 1ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):54 states - 0ms
Total computation time: 3ms.

(min3(n,x))&fac3(i,n,y))):178 states - 2ms
 (z+x)=y:16 states - 0ms
  ((min3(n,x))&fac3(i,n,y)))&(z+x)=y):166 states - 3ms
   (E x , y ((min3(n,x))&fac3(i,n,y)))&(z+x)=y)):41 states - 2ms
Total computation time: 8ms.

(min4(n,x))&fac4(i,n,y))):435 states - 3ms
 (z+x)=y:16 states - 0ms
  ((min4(n,x))&fac4(i,n,y)))&(z+x)=y):407 states - 6ms
   (E

In [73]:
def abeq_fibb8 "?msd_fibb8 $bordercond8(i,j,8,n+7) & (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z)) & (Ez $diff3(i,n,z) &  $diff3(j,n,z)) & (Ez $diff4(i,n,z) &  $diff4(j,n,z)) & (Ez $diff5(i,n,z) &  $diff5(j,n,z)) & (Ez $diff6(i,n,z) &  $diff6(j,n,z)) & (Ez $diff7(i,n,z) &  $diff7(j,n,z)) & (Ez $diff8(i,n,z) &  $diff8(j,n,z))":
def abeq_fib8 "?msd_fib (n<7 & $feq_fib(i,j,n)) | (n>=7 & (?msd_fibb8 Eii,jj,nn ($conv_fib_fibb8(?msd_fib i, ?msd_fibb8 ii) & $conv_fib_fibb8(?msd_fib j, ?msd_fibb8 jj) & $conv_fib_fibb8(?msd_fib n, ?msd_fibb8 nn) & $abeq_fibb8(ii,jj,nn-7))))":

computed ~:1 states - 0ms
computed ~:2 states - 1ms
(diff0(i,n,z))&diff0(j,n,z))):775 states - 4ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):766 states - 5ms
  (bordercond8(i,j,8,(n+7)))&(E z (diff0(i,n,z))&diff0(j,n,z))))):150 states - 2ms
   (diff1(i,n,z))&diff1(j,n,z))):455 states - 3ms
    (E z (diff1(i,n,z))&diff1(j,n,z)))):431 states - 3ms
     ((bordercond8(i,j,8,(n+7)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):146 states - 1ms
      (diff2(i,n,z))&diff2(j,n,z))):327 states - 2ms
       (E z (diff2(i,n,z))&diff2(j,n,z)))):285 states - 2ms
        (((bordercond8(i,j,8,(n+7)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):133 states - 1ms
         (diff3(i,n,z))&diff3(j,n,z))):210 states - 1ms
          (E z (diff3(i,n,z))&diff3(j,n,z)))):195 states - 1ms
           ((((bordercond8(i,j,8,(n+7)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&d

In [74]:
eval comp_fib8 n "?msd_fib Aj $abeq_fib8(i,j,n) => i<=j":

i<=j:6 states - 1ms
 (abeq_fib8(i,j,n))=>i<=j):151 states - 2ms
  (A j (abeq_fib8(i,j,n))=>i<=j)):37 states - 3ms
Total computation time: 6ms.


### Apply the semigroup trick

In [75]:
%SGT comp_fib8 msd_fib Comp_fib8

Reading Result/comp_fib8.mpl
Rank: 11
18 states
values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
Writing Word Automata Library/Comp_fib8.txt
Done.

## Check it out!

In [76]:
%%python
from itertools import product
from ratser.walimp import from_walnut

def valid(a, h, u):
    cur=a
    for c in u:
        v = h[cur]
        if c >= len(v):
            return False
        cur = v[c]
    return True

def enumrepr(a, h, k):
    alpha=list(range(max(map(len,h.values()))))
    for u in product(alpha, repeat=k):
        if valid(a, h, u):
            yield ''.join(map(str,u))

l = []
lbl = []
for k in range(1,9):
    lbl.append(str(k))
    with open(P(os.environ["WALNUT_HOME"]) / P(f"Result/comp_fib{k}.mpl")) as f:
            ser = from_walnut(f)
            l.append(ser)

(a,h)=s.subst()
print(f"{'n':>4}", *[f"{x+'-ab(fib)':>15}" for x in lbl])
print('-'*(4+16*len(lbl)))
for (i, u) in enumerate(enumrepr(a, h, 8)):
    print(f"{i:4}", *[ f"{int(s.value(u)):15}" for s in l ])

   n       1-ab(fib)       2-ab(fib)       3-ab(fib)       4-ab(fib)       5-ab(fib)       6-ab(fib)       7-ab(fib)       8-ab(fib)
------------------------------------------------------------------------------------------------------------------------------------
   0               1               1               1               1               1               1               1               1
   1               2               2               2               2               2               2               2               2
   2               2               3               3               3               3               3               3               3
   3               2               4               4               4               4               4               4               4
   4               2               4               5               5               5               5               5               5
   5               2               4               6               6 

## Let’s wrap it up!

In [77]:
%%shell
cd $WALNUT_HOME
cat << EOF | tr '\n' '\0' | tar cvzf /tmp/abelian.tar.gz --null -T -
Custom Bases/msd_fib.txt
Custom Bases/msd_fib_addition.txt
Word Automata Library/Fib.txt
Automata Library/feq_fib.txt
Result/comp_fib.mpl
Word Automata Library/Comp_fib1.txt
Result/comp_fib1.mpl
Automata Library/abeq_fib1.txt
Word Automata Library/Comp_fib2.txt
Result/comp_fib2.mpl
Automata Library/abeq_fib2.txt
Word Automata Library/Comp_fib3.txt
Result/comp_fib3.mpl
Automata Library/abeq_fib3.txt
Word Automata Library/Comp_fib4.txt
Result/comp_fib4.mpl
Automata Library/abeq_fib4.txt
Word Automata Library/Comp_fib5.txt
Result/comp_fib5.mpl
Automata Library/abeq_fib5.txt
Word Automata Library/Comp_fib6.txt
Result/comp_fib6.mpl
Automata Library/abeq_fib6.txt
Word Automata Library/Comp_fib7.txt
Result/comp_fib7.mpl
Automata Library/abeq_fib7.txt
Word Automata Library/Comp_fib8.txt
Result/comp_fib8.mpl
Automata Library/abeq_fib8.txt
EOF

a Custom Bases/msd_fib.txt
a Custom Bases/msd_fib_addition.txt
a Word Automata Library/Fib.txt
a Automata Library/feq_fib.txt
a Result/comp_fib.mpl
a Word Automata Library/Comp_fib1.txt
a Result/comp_fib1.mpl
a Automata Library/abeq_fib1.txt
a Word Automata Library/Comp_fib2.txt
a Result/comp_fib2.mpl
a Automata Library/abeq_fib2.txt
a Word Automata Library/Comp_fib3.txt
a Result/comp_fib3.mpl
a Automata Library/abeq_fib3.txt
a Word Automata Library/Comp_fib4.txt
a Result/comp_fib4.mpl
a Automata Library/abeq_fib4.txt
a Word Automata Library/Comp_fib5.txt
a Result/comp_fib5.mpl
a Automata Library/abeq_fib5.txt
a Word Automata Library/Comp_fib6.txt
a Result/comp_fib6.mpl
a Automata Library/abeq_fib6.txt
a Word Automata Library/Comp_fib7.txt
a Result/comp_fib7.mpl
a Automata Library/abeq_fib7.txt
a Word Automata Library/Comp_fib8.txt
a Result/comp_fib8.mpl
a Automata Library/abeq_fib8.txt
